In [26]:
import pandas as pd
from pprint import pprint
import numpy as np
import json
from glob import glob
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [32]:
ticker = "AAM"

IS_file = sorted(glob("jsons/IS/" + "*_{}.json".format(ticker)))[0]
BS_file = sorted(glob("jsons/BS/" + "*_{}.json".format(ticker)))[0]
CF_file = sorted(glob("jsons/CF/" + "*_{}.json".format(ticker)))[0]

In [33]:
IS_ACCOUNTS = ["Net sales","Cost of goods sold","Gross Profit","Financial expenses","Of which: Interest expense",
            "Cost of sales","Enterprise cost management","Total Operating Expenses","Total revenue financing activities",
            "Net profit from business activities","Profit","Profit before tax","Present corporate income tax expenses",
            "Deferred income taxes expenses","The interests of minority shareholders","Total Cost of profits",
            "Profit after tax corporate income","Volume","Close of Quarter","EPS","Non-adjusted EPS","PE","Book Price"]

ASSETS = ["SHORT-TERM ASSETS","Cash and cash equivalents","Cash","Cash equivalents","Short-term financial investments",
          "Available for sale securities","Provision for diminution in value of available for sale securities (*)",
          "Held to maturity investments","Short-term receivables","Short-term trade accounts receivable",
          "Short-term prepayments to suppliers","Short-term inter-company receivables","Construction contract progress receipts due from customers",
          "Short-term loan receivables","Other short-term receivables","Provision for short-term doubtful debts (*)",
          "Assets awaiting resolution","Inventories","Inventories","Provision for decline in value of inventories",
          "Other short-term assets","Short-term prepayments","Value added tax to be reclaimed",
          "Taxes and other receivables from state authorities","Government bonds","Other short-term assets","LONG-TERM ASSETS",
          "Long-term receivables","Long-term trade receivables","Long-term prepayments to suppliers","Capital at inter-company",
          "Long-term inter-company receivables","Long-term loan receivables","Other long-term receivables",
          "Provision for long-term doubtful debts","Fixed assets","Tangible fixed assets","Cost","Accumulated depreciation",
          "Financial leased fixed assets","Cost","Accumulated depreciation","Intangible fixed assets","Cost",
          "Accumulated depreciation","Investment properties","Cost","Accumulated depreciation","Long-term assets in progress",
          "Long-term production in progress","Construction in progress","Long-term financial investments",
          "Investments in subsidiaries","Investments in associates, joint-ventures","Investments in other entities",
          "Provision for diminution in value of long-term investments","Held to maturity investments",
          "Other long-term investments","Other long-term assets","Long-term prepayments","Deferred income tax assets",
          "Long-term equipment, supplies, spare parts","Other long-term assets","Goodwill","TOTAL ASSETS"]

LIABILITIES = ["LIABILITIES","Short -term liabilities","Short-term trade accounts payable","Short-term advances from customers",
               "Taxes and other payables to state authorities","Payable to employees","Short-term acrrued expenses",
               "Short-term inter-company payables","Construction contract progress payments due to suppliers",
               "Short-term unearned revenue","Other short-term payables","Short-term borrowings and financial leases",
               "Provision for short-term liabilities","Bonus and welfare fund","Price stabilization fund",
               "Government bonds","Long-term liabilities","Long-term trade payables","Long-term advances from customers",
               "Long-term acrrued expenses","Inter-company payables on business capital","Long-term inter-company payables",
               "Long-term unearned revenue","Other long-term liabilities","Long-term borrowings and financial leases",
               "Convertible bonds","Preferred stock (Debts)","Deferred income tax liabilities","Provision for long-term liabilities",
               "Fund for technology development","Provision for severance allowances"]

EQUITY = ["OWNER'S EQUITY","Owner's equity","Owner's capital","Common stock with voting right","Preferred stock","Share premium",
          "Convertible bond option","Other capital of owners","Treasury shares","Assets revaluation differences",
          "Foreign exchange differences","Investment and development fund","Fund to support corporate restructuring",
          "Other funds from owner's equity","Undistributed earnings after tax","Accumulated retained earning at the end of the previous period",
          "Undistributed earnings in this period","Reserves for investment in construction","Minority's interest",
          "Financial reserves","Other resources and funds","Subsidized not-for-profit funds","Funds invested in fixed assets",
          "MINORITY'S INTEREST","TOTAL OWNER'S EQUITY AND LIABILITIES"]

CF_indirect = ['net_profit_before_tax','adjustments','depreciation_amortization','provisions','net_profit_from_investment_in_joint_venture',
                    'write_off_fixed_assets','unrealised_foreign_exchange_profit','profit_from_disposals_of_fixed_assets',
                    'profit_from_investing_activities','profit_from_deposit','interest_income','interest_expense',
                    'payments_direct_from_profit','operating_profit_before_working_capital_changes','increase_decrease_in_receivables',
                    'increase_decrease_in_inventories','increase_decrease_in_payables','increase_decrease_in_prepaid_expense',
                    'increase_decrease_in_current_assets','cash_paid_for_interest','cash_paid_for_taxes',
                    'other_cash_from_operating_activities','other_payments_from_operating_activities','net_cash_from_operating',
                    'cash_paid_for_new_PPE','cash_collected_from_PPE_sales','cash_paid_for_loans','cash_collected_from_loans',
                    'investment_in_joint_venture','purchases_of_short_term_investment','cash_paid_for_investments_in_other_companies',
                    'cash_collected_from_investments_in_other_companies','interest_collected_from_deposits',
                    'cash_collected_from_interest','purchases_of_minority equity','net_cash_from_investing',
                    'cash_collected_from_issuing_shares','cash_paid_for_capital_contribution','cash_paid_for_short_term_borrowing',
                    'cash_paid_for_principles','cash_paid_for_financial_lease','other_cash_paid_for_financial_activities',
                    'purchase_from_capitalization_issues','dividends_paid','minority_equity_in_joint_venture',
                    'social_welfare_expenses','net_cash_from_financing','net_cash_flow','cash_cash_equivalent_begin',
                    'effects_of_exchange_rate','cash_cash_equivalent_end']

CF_direct = ['cash_collected_from_customers','cash_paid_for_suppliers','cash_paid_for_employees','cash_paid_for_interest',
                  'cash_paid_for_taxes','cash_paid_for_VAT','other_cash_collected_for_operating_activities',
                  'other_cash_paid_for_operating_activities','net_cash_from_operating','cash_paid_for_new_PPE',
                  'cash_collected_from_PPE_sales','cash_paid_for_loans','cash_collected_from_loans',
                  'cash_paid_for_investments_in_other_companies','cash_collected_from_investments_in_other_companies',
                  'cash_collected_from_interest','net_cash_from_investing','cash_collected_from_issuing_shares',
                  'cash_paid_for_capital_contribution','cash_paid_for_short_term_borrowing','cash_paid_for_principles',
                  'cash_paid_for_PPE_lease','cash_paid_for_financial_lease','dividends_paid','cash_paid_for_firm_funds',
                  'net_cash_from_financing','net_cash_flow','cash_cash_equivalent_begin','effects_of_exchange_rate',
                  'cash_cash_equivalent_end']

In [34]:
# GET DATAPOINT FROM AN ACCOUNT, TREATING 'N/A' AS np.nan
def process_array(row, status, account):
    if row[status][account] == 'N/A':
        return np.nan
    else:
        return row[status][account]

In [35]:
IS_df = pd.read_json(IS_file)
IS_df['index'] = np.array([row['quarter'] for row in IS_df['data']])
IS_df.set_index('index', inplace=True)

BS_df = pd.read_json(BS_file)
BS_df['index'] = np.array([row['quarter'] for row in BS_df['data']])
BS_df.set_index('index', inplace=True)

CF_df = pd.read_json(CF_file)
CF_df['index'] = np.array([row['quarter'] for row in CF_df['data']])
CF_df.set_index('index', inplace=True)

# Income statement
for account in IS_ACCOUNTS:
    array = np.array([ process_array(row, 'income status', account) for row in IS_df['data'] ])
    IS_df[account] = array
#     IS_df[account] = (array - np.nanmean(array)) / np.nanstd(array)
IS_df.drop('data', 1, inplace=True)

# Balance sheet
for account in ASSETS:
    array = np.array([ process_array(row, 'assets', account) for row in BS_df['data'] ])
    BS_df[account] = array
#     BS_df[account] = (array - np.nanmean(array)) / np.nanstd(array)
for account in LIABILITIES:
    array = np.array([ process_array(row, 'liabilities', account) for row in BS_df['data'] ])
    BS_df[account] = array
#     BS_df[account] = (array - np.nanmean(array)) / np.nanstd(array)
for account in EQUITY:
    array = np.array([ process_array(row, 'equity', account) for row in BS_df['data'] ])
    BS_df[account] = array
#     BS_df[account] = (array - np.nanmean(array)) / np.nanstd(array)
BS_df.drop('data', 1, inplace=True)

# Cash flow statement
if CF_df['type'][0] == "indirect":
    for account in CF_indirect:
        array = np.array([ process_array(row, 'cash_flow_status', account) for row in CF_df['data'] ])
        CF_df[account] = array
    #     CF_df[account] = (array - np.nanmean(array)) / np.nanstd(array)
else:
    for account in CF_direct:
        array = np.array([ process_array(row, 'cash_flow_status', account) for row in CF_df['data'] ])
        CF_df[account] = array
CF_df.drop('data', 1, inplace=True)

### Combines all 3 DFs

In [41]:
IS_CF = pd.merge(IS_df, CF_df, left_index=True, right_index=True, how='outer')
IS_CF_BS = pd.merge(IS_CF, BS_df, left_index=True, right_index=True, how='outer')

# Reindex
reindex = sorted(IS_CF_BS.index, key = lambda x: int(x.split(" ")[1]))
IS_CF_BS_reindexed = IS_CF_BS.reindex(reindex)
print (reindex)
#IS_CF_BS_reindexed

['Q3 2008', 'Q4 2008', 'Q1 2009', 'Q2 2009', 'Q3 2009', 'Q4 2009', 'Q1 2010', 'Q2 2010', 'Q3 2010', 'Q4 2010', 'Q1 2011', 'Q2 2011', 'Q3 2011', 'Q4 2011', 'Q1 2012', 'Q2 2012', 'Q3 2012', 'Q4 2012', 'Q1 2013', 'Q2 2013', 'Q3 2013', 'Q4 2013', 'Q1 2014', 'Q2 2014', 'Q3 2014', 'Q4 2014', 'Q1 2015', 'Q2 2015', 'Q3 2015', 'Q4 2015', 'Q1 2016', 'Q2 2016', 'Q3 2016', 'Q4 2016', 'Q1 2017']


In [42]:
IS_CF_BS_reindexed['current_ratio'] = IS_CF_BS_reindexed['SHORT-TERM ASSETS'] / IS_CF_BS_reindexed['Short -term liabilities']
IS_CF_BS_reindexed['quick_ratio'] = IS_CF_BS_reindexed[['Cash and cash equivalents', 'Short-term receivables',  'Short-term trade accounts receivable', 'Short-term inter-company receivables', 'Short-term loan receivables', 'Other short-term receivables', 'Taxes and other receivables from state authorities']].sum(axis=1) / IS_CF_BS_reindexed['Short -term liabilities']
IS_CF_BS_reindexed['cash_ratio'] = IS_CF_BS_reindexed['Cash and cash equivalents'] / IS_CF_BS_reindexed['Short -term liabilities']
IS_CF_BS_reindexed['long_term_debt_to_equity'] = IS_CF_BS_reindexed['Long-term liabilities'] / (IS_CF_BS_reindexed['TOTAL OWNER\'S EQUITY AND LIABILITIES'] - IS_CF_BS_reindexed['LIABILITIES'])
IS_CF_BS_reindexed['total_debt_to_equity'] = IS_CF_BS_reindexed['LIABILITIES'] / (IS_CF_BS_reindexed['TOTAL OWNER\'S EQUITY AND LIABILITIES'] - IS_CF_BS_reindexed['LIABILITIES'])
IS_CF_BS_reindexed['debt_ratio'] = IS_CF_BS_reindexed['LIABILITIES'] / IS_CF_BS_reindexed['TOTAL ASSETS']
IS_CF_BS_reindexed['financial_leverage'] = IS_CF_BS_reindexed['TOTAL ASSETS'] / (IS_CF_BS_reindexed['TOTAL OWNER\'S EQUITY AND LIABILITIES'] - IS_CF_BS_reindexed['LIABILITIES'])
IS_CF_BS_reindexed['interest_coverage'] = IS_CF_BS_reindexed[['Profit before tax', 'Of which: Interest expense']].sum(axis=1) / IS_CF_BS_reindexed['Of which: Interest expense']
IS_CF_BS_reindexed['net_profit_margin'] = IS_CF_BS_reindexed['Profit after tax corporate income'] / IS_CF_BS_reindexed['Net sales']
IS_CF_BS_reindexed['operating_profit_margin'] = IS_CF_BS_reindexed[['Profit before tax', 'Of which: Interest expense']].sum(axis=1) / IS_CF_BS_reindexed['Net sales']
IS_CF_BS_reindexed['gross_profit_margin'] = IS_CF_BS_reindexed['Gross Profit'] / IS_CF_BS_reindexed['Net sales']
IS_CF_BS_reindexed['pretax_margin'] = IS_CF_BS_reindexed['Profit before tax'] / IS_CF_BS_reindexed['Net sales']

IS_CF_BS_reindexed['average_total_assets'] = IS_CF_BS_reindexed['TOTAL ASSETS'].rolling(2, min_periods=1).mean()
IS_CF_BS_reindexed['ROA'] = IS_CF_BS_reindexed['Profit after tax corporate income'] / IS_CF_BS_reindexed['average_total_assets']
IS_CF_BS_reindexed['operating_ROA'] = IS_CF_BS_reindexed[['Profit before tax', 'Of which: Interest expense']].sum(axis=1) / IS_CF_BS_reindexed['average_total_assets']

IS_CF_BS_reindexed['average_total_capital'] = IS_CF_BS_reindexed['Owner\'s capital'].rolling(2, min_periods=1).mean()
IS_CF_BS_reindexed['ROC'] = IS_CF_BS_reindexed[['Profit before tax', 'Of which: Interest expense']].sum(axis=1) / IS_CF_BS_reindexed['average_total_capital']

IS_CF_BS_reindexed['average_total_equity'] = (IS_CF_BS_reindexed['TOTAL OWNER\'S EQUITY AND LIABILITIES'] - IS_CF_BS_reindexed['LIABILITIES']).rolling(2, min_periods=1).mean()
IS_CF_BS_reindexed['ROE'] = IS_CF_BS_reindexed['Profit after tax corporate income'] / IS_CF_BS_reindexed['average_total_equity']

IS_CF_BS_reindexed['average_receivables'] = IS_CF_BS_reindexed[['Short-term receivables', 'Short-term trade accounts receivable']].sum(axis=1).rolling(2, min_periods=1).mean()
IS_CF_BS_reindexed['receivables_turnover'] = IS_CF_BS_reindexed['Net sales'] / IS_CF_BS_reindexed['average_receivables']

IS_CF_BS_reindexed['average_inventory'] = IS_CF_BS_reindexed[['Inventories', 'Provision for decline in value of inventories']].sum(axis=1).rolling(2, min_periods=1).mean()
IS_CF_BS_reindexed['inventory_turnover'] = IS_CF_BS_reindexed['Cost of goods sold'] / IS_CF_BS_reindexed['average_inventory']

IS_CF_BS_reindexed['average_payables'] = IS_CF_BS_reindexed['Short-term trade accounts payable'].rolling(window=2, min_periods=1).mean()
IS_CF_BS_reindexed['payables_turnover'] = (IS_CF_BS_reindexed[['Inventories', 'Provision for decline in value of inventories']].sum(axis=1).diff(1).fillna(0) + IS_CF_BS_reindexed['Cost of goods sold']) / IS_CF_BS_reindexed['average_payables']

IS_CF_BS_reindexed['book_value_per_share'] = (IS_CF_BS_reindexed['TOTAL OWNER\'S EQUITY AND LIABILITIES'] - IS_CF_BS_reindexed['LIABILITIES']) / IS_CF_BS_reindexed['Volume']
IS_CF_BS_reindexed['cash_ROA'] = IS_CF_BS_reindexed['net_cash_from_operating'] / IS_CF_BS_reindexed['average_total_assets']
IS_CF_BS_reindexed['cash_flow_on_revenue'] = IS_CF_BS_reindexed['net_cash_from_operating'] / IS_CF_BS_reindexed['Net sales']
IS_CF_BS_reindexed['cash_ROE'] = IS_CF_BS_reindexed['net_cash_from_operating'] / IS_CF_BS_reindexed['average_total_equity']
IS_CF_BS_reindexed['cash_on_income'] = IS_CF_BS_reindexed['net_cash_from_operating'] / IS_CF_BS_reindexed[['Profit before tax', 'Of which: Interest expense']].sum(axis=1)
IS_CF_BS_reindexed['debt_coverage'] = IS_CF_BS_reindexed['net_cash_from_operating'] / IS_CF_BS_reindexed['LIABILITIES']
IS_CF_BS_reindexed['interest_coverage'] = IS_CF_BS_reindexed[['net_cash_from_operating', 'cash_paid_for_interest', 'cash_paid_for_taxes']].sum(axis=1) / IS_CF_BS_reindexed['cash_paid_for_interest']
IS_CF_BS_reindexed['reinvestment'] = IS_CF_BS_reindexed['net_cash_from_operating'] / IS_CF_BS_reindexed['cash_paid_for_new_PPE']
IS_CF_BS_reindexed['total_net_accruals'] = IS_CF_BS_reindexed['TOTAL ASSETS'].diff(1).fillna(0) - IS_CF_BS_reindexed['LIABILITIES'].diff(1).fillna(0) - IS_CF_BS_reindexed['net_cash_flow']
IS_CF_BS_reindexed['cash_earnings'] = IS_CF_BS_reindexed['total_net_accruals'] - IS_CF_BS_reindexed['Profit after tax corporate income']

In [43]:
# For the moment just ignore this part
# corr_matrix = IS_CF_BS_reindexed.iloc[:, 188:220].corr().abs()
# corr_matrix_diag = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# sorted_corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)).stack().sort_values(ascending=False)
# sorted_corr_matrix[sorted_corr_matrix>0.8]

In [44]:
bg = IS_CF_BS_reindexed.columns.get_loc('current_ratio')
end = IS_CF_BS_reindexed.columns.get_loc('cash_earnings')+1
indexes_df = IS_CF_BS_reindexed.iloc[:, bg:end]
indexes_df

,current_ratio,quick_ratio,cash_ratio,long_term_debt_to_equity,total_debt_to_equity,debt_ratio,financial_leverage,interest_coverage,net_profit_margin,operating_profit_margin,...,payables_turnover,book_value_per_share,cash_ROA,cash_flow_on_revenue,cash_ROE,cash_on_income,debt_coverage,reinvestment,total_net_accruals,cash_earnings
index,,,,,,,,,,,,,,,,,,,,,
Q3 2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009962,0.010767,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q4 2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.138224,-0.143318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1 2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087555,0.095253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q2 2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149600,0.185157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q3 2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.093647,0.104217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q4 2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.134443,0.164810,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1 2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.114642,0.133909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q2 2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122881,0.145714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q3 2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.066533,0.077809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
indexes_df_normalized = (indexes_df - indexes_df.mean()) / indexes_df.std()
corr_matrix1 = indexes_df_normalized.corr().abs()
corr_matrix_diag1 = corr_matrix1.where(np.triu(np.ones(corr_matrix1.shape), k=1).astype(np.bool))
sorted_corr_matrix_diag1 = corr_matrix_diag1.stack()
# .sort_values(ascending=False)
# sorted_corr_matrix_diag1[sorted_corr_matrix_diag1>0.8]
# corr_matrix1
corr_matrix_diag1

,current_ratio,quick_ratio,cash_ratio,long_term_debt_to_equity,total_debt_to_equity,debt_ratio,financial_leverage,interest_coverage,net_profit_margin,operating_profit_margin,...,payables_turnover,book_value_per_share,cash_ROA,cash_flow_on_revenue,cash_ROE,cash_on_income,debt_coverage,reinvestment,total_net_accruals,cash_earnings
current_ratio,NaN,0.918021,0.033101,0.211133,0.817078,0.848229,0.817078,0.208362,0.244754,0.124982,...,0.160863,0.203198,0.118203,0.075740,0.110997,0.153599,0.207314,0.129060,0.000694,0.105172
quick_ratio,NaN,NaN,0.282048,0.112048,0.774204,0.797575,0.774204,0.185738,0.109473,0.121946,...,0.272034,0.033680,0.015798,0.016640,0.009566,0.104173,0.124688,0.173041,0.062839,0.088149
cash_ratio,NaN,NaN,NaN,0.199417,0.295886,0.278417,0.295886,0.125047,0.314594,0.153134,...,0.513200,0.349275,0.231355,0.190443,0.230395,0.086790,0.224534,0.122444,0.328302,0.423951
long_term_debt_to_equity,NaN,NaN,NaN,NaN,0.276416,0.281356,0.276416,0.131426,0.905059,0.797546,...,0.038674,0.636093,0.029809,0.003292,0.033131,0.036846,0.054174,0.161414,0.230027,0.134650
total_debt_to_equity,NaN,NaN,NaN,NaN,NaN,0.997951,1.000000,0.201981,0.238178,0.096378,...,0.358570,0.008332,0.090178,0.124506,0.095313,0.001812,0.046134,0.027386,0.170114,0.052456
debt_ratio,NaN,NaN,NaN,NaN,NaN,NaN,0.997951,0.210534,0.248656,0.108472,...,0.349107,0.024466,0.073043,0.109633,0.078151,0.010491,0.027864,0.010884,0.159376,0.038621
financial_leverage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.201981,0.238178,0.096378,...,0.358570,0.008332,0.090178,0.124506,0.095313,0.001812,0.046134,0.027386,0.170114,0.052456
interest_coverage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038888,0.107545,...,0.370437,0.213901,0.612550,0.598279,0.616447,0.521266,0.558984,0.409478,0.354524,0.325085
net_profit_margin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.722254,...,0.234737,0.580698,0.192423,0.205322,0.190433,0.087964,0.152661,0.083032,0.060472,0.319213
operating_profit_margin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.007874,0.547489,0.119098,0.072722,0.128709,0.033570,0.111743,0.106157,0.348703,0.080080
